# 월간 데이콘 법원 판결 예측 AI 경진대회
This code come from [here](https://dacon.io/competitions/official/236112/codeshare/8425?page=1&dtype=recent).

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [4]:
train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [8]:
print(train['facts'][0])

On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana.  During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union.  Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. 
Thompson successfully sued St. Amant for defamation.  Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.”  Thompson then appealed to the Supreme Court of Louisiana.  That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true.  Finally, that court held that the First Amendment protects uninhibited, robust debate, rather tha

In [7]:
print(train['facts'][1])

Ramon Nelson was riding his bike when he suffered a lethal blow to the back of his head with a baseball bat. After two eyewitnesses identified Lawrence Owens from an array of photos and then a lineup, he was tried and convicted for Nelson’s death. Because Nelson was carrying cocaine and crack cocaine potentially for distribution, the judge at Owens’ bench trial ruled that Owens was probably also a drug dealer and was trying to “knock [Nelson] off.” Owens was found guilty of first-degree murder and sentenced to 25 years in prison.
Owens filed a petition for a writ of habeas corpus on the grounds that his constitutional right to due process was violated during the trial. He argued that the eyewitness identification should have been inadmissible based on unreliability and that the judge impermissibly inferred a motive when a motive was not an element of the offense. The district court denied the writ of habeas corpus, and Owens appealed. The U.S. Court of Appeals for the Seventh Circuit r

# Data Preprocessing

In [10]:
vectorizer = TfidfVectorizer()

def get_vector(vectorizer, df, train_mode):

    if train_mode:
        X_facts = vectorizer.fit_transform(df['facts']) # fit and transform
    else:
        X_facts = vectorizer.transform(df['facts']) # only transform

    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])

    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
    return X

In [11]:
X_train = get_vector(vectorizer, train, True)
Y_train = train['first_party_winner']
X_test = get_vector(vectorizer, test, False)

In [17]:
X_train = np.array(X_train)
X_test = np.array(X_test)

# Define Model & Train

In [16]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

# Inference & Submission

In [19]:
submit = pd.read_csv('../data/sample_submission.csv')

In [20]:
pred = model.predict(X_test)

In [21]:
pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [22]:
submit['first_party_winner'] = pred
submit.to_csv('./baseline_submit.csv', index=False)